# dspy playbook

[DSYp](https://dspy.ai/) is:
> a declarative framework for building modular AI software. It allows you to iterate fast on structured code, rather than brittle strings, and offers algorithms that compile AI programs into effective prompts and weights for your language models, whether you're building simple classifiers, sophisticated RAG pipelines, or Agent loops.

As such, it shifts your focus from fiddling with prompt strings to programming with structured and declarative natural-language modules. 

### Configuration

In [1]:
import os
import dspy

lm = None

openai_key = os.environ.get('OPENAI_API_KEY')
anthropic_key = os.environ.get('ANTHROPIC_API_KEY')
perplexity_key = os.environ.get('PERPLEXITY_API_KEY')
groq_key = os.environ.get('GROQ_API_KEY')

if None in [openai_key,anthropic_key,perplexity_key,groq_key]:
    raise ValueError('No supported API key found in environment variables.')

if openai_key:
    openai_lm = dspy.LM('openai/gpt-4o-mini', api_key=openai_key)
if anthropic_key:
    anthropic_lm = dspy.LM('anthropic/claude-opus-4-20250514', api_key=anthropic_key)
if perplexity_key:
    perplexity_lm = dspy.LM('perplexity/sonar', api_key=perplexity_key)
if groq_key:
    groq_lm = dspy.LM('groq/llama-3.1-8b-instant', api_key=groq_key)

dspy.configure(lm=openai_lm)

### Use Case 1: Information Extraction 

In [2]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today.  The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)

Apple Inc. Announces iPhone 14
['Announcement', 'CEO Statement', 'New Features']
[{'name': 'Apple Inc.', 'type': 'Company'}, {'name': 'iPhone 14', 'type': 'Product'}, {'name': 'Tim Cook', 'type': 'Person'}]


### Use Case 2: Prediction 

In [3]:
from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal["positive", "negative", "neutral"] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read, though not the last chapter.")

Prediction(
    sentiment='positive',
    confidence=0.85
)

### Use Case 3: RAG

In [4]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

rag = dspy.ChainOfThought("context, question -> response")

question = "What's the name of the castle that David Gregory inherited?"
rag(context=search_wikipedia(question), question=question)

Prediction(
    reasoning='David Gregory inherited Kinnairdy Castle in 1664, as mentioned in the context provided.',
    response='The name of the castle that David Gregory inherited is Kinnairdy Castle.'
)

### Use Case 4: ReAct-based Agents

[ReAct]() is a pattern for 

In [5]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

target = 9362158
pred1 = react(question="What is the year of birth of David Gregory of Kinnairdy castle?")
print(pred1.answer)

1625.0


In [6]:
pred2 = react(question=f"What is {target} divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred2.answer)
print(f"Expected answer is {target/pred1.answer}")

5765.0
Expected answer is 5761.328
